<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335206/logos/Logo_des_Forschungszentrums_J_C3_BClich_seit_2018_hcliq4.svg"  width=250 align='left' style="margin-top:30px"/>
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637333672/logos/ebrains_logo_swnlzv.png" width="280" align='left' style="margin-left:50px; margin-top:25px">
</div>  
<br><br><br><br>
<br>

# Exploring SSB pathways associated to disease variants 

## The OXTR pathway as a study case

In this notebook, we will simulate a mathematical model of the signaling pathway of the Oxytocin receptor. This will provide a rationale for experimental observations on a disease-associated variant of the OXT receptor.

The data used in this notebook is based on:
<blockquote>
Meyer, M., Jurek, B., Alfonso-Prieto, M. et al. Structure-function relationships of the 
disease-linked A218T oxytocin receptor variant. Mol Psychiatry (2022). 
<a href="https://doi.org/10.1038/s41380-021-01241-8" target="_bank"> https://doi.org/10.1038/s41380-021-01241-8</a>
</blockquote>

This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

<div style='padding:15px'>
<a href="https://colab.research.google.com/github/rribeiro-sci/SSBtoolkit/blob/main/Tutorials/SSBtoolkit-Tutorial4-OXTR.ipynb" target="_blank">
<img alt="Colab" src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637335713/badges/colab-badge_hh0uyl.svg" height="25" style="margin:20px">
</a>




> ⚠️ **Important:**  
> If you are working within the **Google Colaboratory** environment, please **change the Colab runtime version** to avoid potential version compatibility issues.  
>
> **To update the runtime version:**
> 1. Go to the **“Runtime”** menu at the top of Colab.  
> 2. Select **“Change runtime type.”**  
> 3. From the **“Runtime version”** dropdown, choose **`2027.07`**.  
>


## Installing the SSB toolkit library

In [ ]:
%%capture
!pip install --quiet ssbtoolkit
import os
os.kill(os.getpid(), 9)  # Restart the kernel to load the newly installed package

### Importing dependencies

In [ ]:
#Import Python dependencies
import numpy as np
import pandas as pd
import ssbtoolkit as ssb

In [ ]:
from plotly import io as pio
pio.renderers.default = 'colab'

## Introduction

The neuropeptide oxytocin (OXT) regulates multiple social and emotional behaviors, such as bonding, reciprocal trust, aggression, fear, and anxiety, in both animals and humans. It has been suggested as a biomarker and target for the treatment of Autism Spectrum Disorder (ASD)  [(Mayer et al. 2022)](https://doi.org/10.1038/s41380-021-01241-8).

OXT exerts its function by binding to its target receptor (OXTR), a member of the Class A G-protein coupled receptors (GPCR) family.

A mutation of an alanine residue at position 218 to a threonine (A218T) in the OXTR has been associated with a phenotype related to ASD.

In vitro experiments have shown differences in the dynamics of intracellular Ca<sup>2+</sup> between the wild-type OXTR (OXTR-WT) and the OXTR-A218T variant.


To better understand these differences, we will use the SSB Toolkit to fit a mathematical model of the OXTR signaling pathway to the observed experimental results.

Since we aim to simulate and study the dynamics of Ca<sup>2+</sup> release from the endoplasmic reticulum, which is triggered by the G<sub>q</sub> pathway (to which OXTR couples), we will use a combination of two previously existing models: one for the activation of the G<sub>q</sub> protein and another for the dynamic model of IP<sub>3</sub>R action. See [Mayer et al. 2022](https://doi.org/10.1038/s41380-021-01241-8) for all the details.

## Experimental Data

<blockquote>n vitro evaluation of cellular Ca<sup>2+</sup> responses using Fura-2 Ca<sup>2+</sup> imaging (see figures a and b) revealed reduced basal cytosolic Ca<sup>2+</sup> levels in A218T cells compared to WT, both in the presence and absence of extracellular Ca<sup>2+</sup> (figure <b>c</b>). 

However, the amplitude of the OXT-induced Ca<sup>2+</sup> signal was higher in A218T cells compared to WT cells incubated in Ca<sup>2+</sup>-free Ringer (figure <b>d</b>). 
        
The area under the curve (AUC) showed a cell line-specific effect, indicating a higher increase in cytosolic Ca<sup>2+</sup> concentration upon OXT stimulation in A218T cells compared to WT cells (figure <b>e</b>). 
    
The full width at half maximum (FWHM), reflecting the kinetics of the OXT-induced Ca<sup>2+</sup> response, also differed significantly between the two cell lines, irrespective of the bathing solution. This indicates a prolonged OXT-induced Ca<sup>2+</sup> response in A218T cells (figure <b>f</b>).


<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1640172296/SSBColab/OXT_results_vsvpns.png" width=600/>
</div>  

</blockquote>




## Modeling of the Ca<sup>2+</sup> concentration differences between WT and A218T OXTR

Our mail goal is to use the <b>SSB toolkit</b> to rationalize the experimentally measured Ca<sup>2+</sup> concentrations with 100 nM OXT in the absence of extracellular Ca<sup>2+</sup> (<b>figure e, blue bars</b>). 

Molecular modeling results have suggested that the A218T variant affects receptor activation (see [Mayer et al. 2022](https://doi.org/10.1038/s41380-021-01241-8) for details). This implies that the variant may influence the kinetic parameters governing the receptor activation reaction. 

Therefore we will simulate the OXTR signalling model by varying the forward kinetic constant of the reaction that describes Gq-protein binding to the receptor, which implicitly depends on receptor activation. 

To achieve this, we will use the SSB Toolkit’s built-in function `Simulation.FitModel()`. This function allows us to iteratively simulate the model until we obtain the desired ratio of the concentration of a signaling species between a model simulated with modified parameters and a model simulated with reference parameters.

First, we need to set up the following parameters:
* **fitting parameters**
* **simulation parameters**
* **pathway parameters**


#### Fitting Parameters

* expratio (flt)          --> experimental signalling specie concentration ratio
* target_parameter (str)  --> kinetic parameter to be modified
* maxiter  (int)          --> maximum number of iterations
* seed     (flt)          --> ramdom seed for scaling the modified parameter
* seed_incrementor (flt)  --> seed incrementor (each iteration will increment the seed by this value)
* seed_decrementor (flt)  --> seed decrementor (each iteration will decrement the seed by this value)


#### Simulation Parameters

* ttotal   (int)     --> time of simulation in seconds
* nsteps   (int)     --> number of time steps
* pathway (str)      --> name of the signalling pathway 
* observable (str)   --> signalling specie to be measured

#### Pahtway Parameters

* time_in  (int)     --> simulation time at wich the equation will start to be simulated 
* time_out (int)     --> simulation time at wich the equation will stop to be simulated

In [ ]:
# we will start by creating a simulation instance.
sim = ssb.Simulation.FitModel()

In [ ]:
# configuration of simulation parameters
sim.SetSimulationParameters(ttotal=250, 
                            nsteps=10000, 
                            pathway='OXTR_pathway', 
                            observable='Ca', 
                            pathway_parameters={'time_in':50, 
                                                'time_out':51})

In [ ]:
# Starting the simulation
sim.Run(expratio=1.13, 
        target_parameter='R_L_Gq_trimer_kf',
        maxiter=20, 
        seed=0.2, 
        seed_incrementor=0.5, 
        seed_decrementor=0.1)

Now, we can plot the curves, amplitudes, area under the curve (AUC), and the full width at half maximum (FWHM):

In [ ]:
sim.ShowGraphs()

## Congratulations! 

Congratulations on completing this tutorial notebook! If you enjoyed working through it and would like to continue exploring the SSB Toolkit, we encourage you to complete the rest of the tutorials in this series.

## Cite Us

If you use or adapt this tutorial for your own research projects please cite us.

```
@article{ribeiro_ssb_2022,
    title={Structural Systems Biology Toolkit (SSBtoolkit): From Molecular Structure to Subcellular Signaling Pathways},
    author={Ribeiro, Rui Pedro and Gossen, Jonas and Rossetti, Giulia and Giorgetti, Alejandro},
    publisher={J. Chem. Inf. Model.},
    url={https://doi.org/10.1021/acs.jcim.5c00165},
    year={2025}
}
```


## Acknowledgments

EU Human Brain Project (SGA1 and SGA2): This open source software was developed in part in the Human Brain Project funded from the European Union's Horizon 2020 Framework Programme for Research and Innovation under Specific Grant Agreements No 720270 and No. 78907 (Human Brain Project SGA1 and SGA2).

<div style="padding-bottom:50px">
<img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1637657234/logos/HBP_horizontal_logo_qtcyzn.png" width="300" align='left' style="margin-left:50px">
    <img src="https://res.cloudinary.com/djz27k5hg/image/upload/v1642677502/logos/COFUNDED_EU_j2ktlp.jpg" width="300" align='left' style="margin-left:50px">
</div>  